### Configurações Pyspark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window

import pandas as pd

def write_to_postgresl(df, tb_name=None, write_mode='None'):
    df.count()
    if tb_name is None:
        raise Exception('Informe o nome da tabela')
    if write_mode is None:
        raise Exception('Informe o mode de escrita: append ou overwrite')
        
    from datetime import datetime
    start_time = datetime.now()
    
    try:
        df.write.jdbc(url, table=tb_name, mode=write_mode, properties=properties)
    except Exception as e:
        print(f'Erro: {e}')
    end_time = datetime.now()
    
    total_time = str(end_time - start_time)
    
    return f'Total time: {total_time} and - Total rows: {df.count()} - Total columns: {len(df.columns)}'



    # Criando a sessão do Spark
spark  = SparkSession.builder \
    .appName("Data Analysis") \
    .config('spark.jars', '/data/IDAF/DATABASECONNECTOR_JAR_FOLDER/postgresql-42.2.18.jar')\
    .config("spark.executor.memory", "8g")\
    .config("spark.driver.memory", "8g")\
    .config("spark.executor.cores", "4")\
    .config("spark.executor.instances","8")\
    .config("spark.sql.shuffle.partitions","96")\
    .config("spark.default.parallelism","96")\
    .getOrCreate()


url = "jdbc:postgresql://localhost:5432/postgres"

properties = {
    "user" : "postgres",
    "password" : "cidacs",
    "driver" : "org.postgresql.Driver"
}

25/07/20 17:10:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Lendo dados enriquecidos

In [2]:
df_input = (spark
            .read
            .parquet('/data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/scripts_omop_tb/basefinal_tb_limp_enriched')
           )

### Location

In [3]:
observation_concept_list = ['tp_not_concept', 'situa_ence_concept', 'dt_diag_concept', 'escmae_sinasc_concept',
                            'dtobito_sim_y_concept', 'estcivmae_sinasc_concept', 'qtdfilvivo_sinasc_concept', 'qtdfilmort_sinasc_concept',
                            'consprenat_sinasc_concept', 'qtdgestant_concept', 'tratamento_concept', 'benef_gov_concept', 'situa_9_m_concept']

df_observation = (df_input
                  .withColumn('tp_not_concept', F.when(F.col('`tp_not.y`').isNotNull(), F.lit(4015925)).otherwise(None).cast('int'))
                  .withColumn('situa_ence_concept', F.when(F.col('situa_ence')==0, F.lit(4255047))
                                                     .when(F.col('situa_ence')==1, F.lit(4159637))
                                                     .when(F.col('situa_ence')==2, F.lit(4160623))
                                                     .when(F.col('situa_ence')==3, F.lit(4306655))
                                                     .when(F.col('situa_ence')==4, F.lit(441413))
                                                     .when(F.col('situa_ence')==5, F.lit(4057377))
                                                     .when(F.col('situa_ence')==6, F.lit(4049904))
                                                     .when(F.col('situa_ence')==7, F.lit(4005145))
                                                     .when(F.col('situa_ence')==8, F.lit(762851))
                                                     .when(F.col('situa_ence')==9, F.lit(4023307))
                                                     .when(F.col('situa_ence')==10, F.lit(4160623))
                                                     .when(F.col('situa_ence')==99, F.lit(4127662))
                                                     .otherwise(None).cast('int'))
                  .withColumn('dt_diag_concept', F.when(F.col('dt_diag').isNotNull(), F.lit(4160852)).otherwise(None).cast('int'))
                  .withColumn('escmae_sinasc_concept', F.when(F.col('escmae_sinasc').isNotNull(), F.lit(4144287)).otherwise(None).cast('int'))
                  .withColumn('dtobito_sim_y_concept', F.when(F.col('dtobito_sim').isNotNull(), F.lit(4244279)).otherwise(None).cast('int'))
                  .withColumn('estcivmae_sinasc_concept', F.when(F.col('estcivmae_sinasc').isNotNull(), F.lit(4053609)).otherwise(None).cast('int'))
                  .withColumn('qtdfilvivo_sinasc_concept', F.when(F.col('qtdfilvivo_sinasc').isNotNull(), F.lit(45770046)).otherwise(None).cast('int'))
                  .withColumn('qtdfilmort_sinasc_concept', F.when(F.col('qtdfilmort_sinasc').isNotNull(), F.lit(4090055)).otherwise(None).cast('int'))
                  .withColumn('consprenat_sinasc_concept', F.when(F.col('consprenat_sinasc').isNotNull(), F.lit(4313474)).otherwise(None).cast('int'))
                  .withColumn('qtdgestant_concept', F.when(F.col('qtdgestant_sinasc').isNotNull(), F.lit(4132561)).otherwise(None).cast('int'))
                  .withColumn('tratamento_concept', F.when(F.col('tratamento').isNotNull(), F.lit(4086442)).otherwise(None).cast('int'))
                  .withColumn('benef_gov_concept', F.when(F.col('benef_gov').isNotNull(), F.lit(44804295)).otherwise(None).cast('int'))
                  .withColumn('situa_9_m_concept', F.when(F.col('situa_9_m').isNotNull(), F.lit(45955607)).otherwise(None).cast('int'))
                  .withColumn('observation_concept_id_list', F.split(F.concat_ws(',', *observation_concept_list), ','))
                  .withColumn('observation_concept_id', F.explode('observation_concept_id_list'))
                  .withColumn('observation_date', F.when(F.col('observation_concept_id')==4015925, F.col('`dt_notific.y`'))
                                                   .when(F.col('observation_concept_id')==4086442, F.col('dt_inic_tr'))
                                                   .when(F.col('observation_concept_id').isin(4255047, 4159637, 4160623, 4306655, 441413, 4057377, 4049904, 4005145, 762851, 4023307, 4160623, 4127662), F.col('dt_inic_tr'))
                                                   .when(F.col('observation_concept_id').isin(4144287, 4053609, 4132561, 4090055, 4313474), F.col('dtnasc_sinasc'))
                                                   .when(F.col('observation_concept_id').isin(4160852), F.col('dt_diag'))
                                                   .when(F.col('observation_concept_id').isin(4244279), F.col('dtobito_sim'))
                                                   .otherwise(F.col('dtnasc_sinasc'))
                             )
                  .withColumn('observation_datetime', F.lit(None).cast('timestamp'))
                  .withColumn('qtdfilvivo_sinasc_calc', F.col('qtdfilvivo_sinasc').cast('int') + F.col('qtdfilmort_sinasc').cast('int'))
                  .withColumn('qtdfilmort_sinasc_calc', F.col('qtdfilmort_sinasc').cast('int'))
                  .withColumn('consprenat_sinasc_calc', F.col('consprenat_sinasc').cast('int'))
                  .withColumn('qtdgestant_sinasc_calc', F.col('consprenat_sinasc').cast('int'))
                  .withColumn('value_as_number_list', F.split(F.concat_ws(',', 'qtdgestant_sinasc_calc', 'qtdfilvivo_sinasc_calc', 'qtdfilmort_sinasc_calc', 'consprenat_sinasc_calc'), ','))
                  .withColumn('value_as_number', F.explode(F.col('value_as_number_list')))
                  .withColumn('value_as_number', 
                             F.when(F.col('observation_concept_id')==45770046, F.col('qtdfilvivo_sinasc'))
                              .when(F.col('observation_concept_id')==4090055, F.col('qtdfilmort_sinasc'))
                              .when(F.col('observation_concept_id')==4313474, F.col('consprenat_sinasc'))
                              .when(F.col('observation_concept_id')==4132561, F.col('qtdgestant_sinasc'))
                              .otherwise(None))
                  .withColumn('tp_not_calc', 
                             F.when((F.col('tp_not_concept').isNotNull())&(F.col('`tp_not.y`')==0), F.lit(None))
                              .when((F.col('tp_not_concept').isNotNull())&(F.col('`tp_not.y`')==1), F.lit(9189))
                              .when((F.col('tp_not_concept').isNotNull())&(F.col('`tp_not.y`')==2), F.lit(45879974))
                              .when((F.col('tp_not_concept').isNotNull())&(F.col('`tp_not.y`')==3), F.lit(40482790))
                              .when((F.col('tp_not_concept').isNotNull())&(F.col('`tp_not.y`')==4), F.lit(4083580))
                              .when((F.col('tp_not_concept').isNotNull())&(F.col('`tp_not.y`')==5), F.lit(4127662))
                              .otherwise(None)
                             )
                  .withColumn('tratamento_calc', 
                             F.when((F.col('tratamento_concept').isNotNull())&(F.col('tratamento')==0), F.lit(4080894))
                              .when((F.col('tratamento_concept').isNotNull())&(F.col('tratamento')==1), F.lit(4089048))
                              .when((F.col('tratamento_concept').isNotNull())&(F.col('tratamento')==2), F.lit(4078193))
                              .when((F.col('tratamento_concept').isNotNull())&(F.col('tratamento')==3), F.lit(45880829))
                              .when((F.col('tratamento_concept').isNotNull())&(F.col('tratamento')==4), F.lit(4160775))
                              .when((F.col('tratamento_concept').isNotNull())&(F.col('tratamento')==5), F.lit(4030023))
                              .when((F.col('tratamento_concept').isNotNull())&(F.col('tratamento')==6), F.lit(4046028))
                              .when((F.col('tratamento_concept').isNotNull())&(F.col('tratamento')==99), F.lit(4127662))
                              .otherwise(0))
                  .withColumn('situa_9_m_calc', 
                             F.when((F.col('situa_9_m_concept').isNotNull())&(F.col('situa_9_m')==0), F.lit(None))
                              .when((F.col('situa_9_m_concept').isNotNull())&(F.col('situa_9_m')==1), F.lit(4159637))
                              .when((F.col('situa_9_m_concept').isNotNull())&(F.col('situa_9_m')==2), F.lit(4160623))
                              .when((F.col('situa_9_m_concept').isNotNull())&(F.col('situa_9_m')==3), F.lit(4306655))
                              .when((F.col('situa_9_m_concept').isNotNull())&(F.col('situa_9_m')==4), F.lit(441413))
                              .when((F.col('situa_9_m_concept').isNotNull())&(F.col('situa_9_m').isin(5,6,7,8)), F.lit(4057377))
                              .when((F.col('situa_9_m_concept').isNotNull())&(F.col('situa_9_m')==9), F.lit(4253173))
                              .when((F.col('situa_9_m_concept').isNotNull())&(F.col('situa_9_m')==10), F.lit(4049904))
                              .when((F.col('situa_9_m_concept').isNotNull())&(F.col('situa_9_m')==11), F.lit(4023307))
                              .when((F.col('situa_9_m_concept').isNotNull())&(F.col('situa_9_m')==12), F.lit(44789054))
                              .when((F.col('situa_9_m_concept').isNotNull())&(F.col('situa_9_m')==13), F.lit(3043504))
                              .when((F.col('situa_9_m_concept').isNotNull())&(F.col('situa_9_m')==99), F.lit(4088484))
                              .otherwise(0))
                  .withColumn('value_as_concept_id_list', F.split(F.concat_ws(',', 'tp_not_calc', 'tratamento_calc', 'situa_9_m_calc'), ','))
                  .withColumn('value_as_concept_id', F.explode('value_as_concept_id_list'))
                  .withColumn('observation_type_concept_id', F.lit(32879))
                  .withColumn('value_as_string', F.when((F.col('observation_concept_id')==44804295)&(F.col('benef_gov')==0), F.lit(None))
                                                  .when((F.col('observation_concept_id')==44804295)&(F.col('benef_gov')==1), F.lit(4188539))
                                                  .when((F.col('observation_concept_id')==44804295)&(F.col('benef_gov')==2), F.lit(4188540))
                                                  .when((F.col('observation_concept_id')==44804295)&(F.col('benef_gov')==88), F.lit(4118638))
                                                  .when((F.col('observation_concept_id')==44804295)&(F.col('benef_gov')==99), F.lit(4127662))
                                                  .otherwise(None)
                             )
                  .withColumn('qualifier_concept_id', F.lit(None))
                  .withColumn('unit_concept_id', F.lit(None))
                  .withColumn('provider_id', F.lit(None))
                  .withColumn('visit_occurrence_id', F.col('visit_occurrence_id'))
                  .withColumn('visit_detail_id', F.lit(None))
                  .withColumn('observation_source_value', F.lit(None))
                  .withColumn('observation_source_concept_id', F.lit(None))
                  .withColumn('unit_source_value', F.lit(None))
                  .withColumn('qualifier_source_value', F.lit(None))
                  .withColumn('value_source_value', F.lit(None))
                  .withColumn('observation_event_id', F.lit(None))
                  .withColumn('obs_event_field_concept_id', F.lit(None))
                  .withColumn('observation_id', F.row_number().over(Window.orderBy('person_id', 'dtnasc_sinasc')))
                  .filter(F.col('observation_concept_id').isNotNull())
                  .filter(F.col('observation_date').isNotNull())
                             ).select(
                                 F.col('observation_id').cast('integer'),
                                 F.col('person_id').cast('integer'),
                                 F.col('observation_concept_id').cast('integer'),
                                 F.col('observation_date').cast('date'),
                                 F.col('observation_datetime').cast('timestamp'),
                                 F.col('observation_type_concept_id').cast('integer'),
                                 F.col('value_as_number').cast('float'),
                                 F.col('value_as_string').cast('string'),
                                 F.col('value_as_concept_id').cast('integer'),
                                 F.col('qualifier_concept_id').cast('integer'),
                                 F.col('unit_concept_id').cast('integer'),
                                 F.col('provider_id').cast('integer'),
                                 F.col('visit_occurrence_id').cast('integer'),
                                 F.col('visit_detail_id').cast('integer'),
                                 F.col('observation_source_value').cast('string'),
                                 F.col('observation_source_concept_id').cast('integer'),
                                 F.col('unit_source_value').cast('string'),
                                 F.col('qualifier_source_value').cast('string'),
                                 F.col('value_source_value').cast('string'),
                                 F.col('observation_event_id').cast('integer'),
                                 F.col('obs_event_field_concept_id').cast('integer')
                            ).distinct()
           
# df_observation.count()

In [4]:
df_observation.repartition(15).write.parquet('/data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/scripts_omop_tb/basefinal_tb_limp_enriched/observation_batch')

25/07/20 17:10:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/07/20 17:10:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/07/20 17:10:19 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/07/20 17:12:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/07/20 17:12:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/07/20 17:16:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:16:47 WARN RowBasedKey

25/07/20 17:22:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:22:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:22:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:22:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:23:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:23:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:23:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:23:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:23:17 WARN RowBasedKeyValueBatch: Calling spill() on

25/07/20 17:26:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:26:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:26:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:26:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:27:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:27:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:27:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:27:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:27:14 WARN RowBasedKeyValueBatch: Calling spill() on

25/07/20 17:30:45 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:30:45 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:30:45 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:30:45 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:30:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:30:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:30:59 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:30:59 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:31:12 WARN RowBasedKeyValueBatch: Calling spill() on

25/07/20 17:35:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:35:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:35:27 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:35:27 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:35:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:35:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:36:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:36:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/07/20 17:36:49 WARN RowBasedKeyValueBatch: Calling spill() on

In [5]:
df_1 = spark.read.parquet('/data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/scripts_omop_tb/basefinal_tb_limp_enriched/observation_batch')

In [6]:
df_1.count()

723992343

In [4]:
df_1.repartition(10).write.parquet('/data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/scripts_omop_tb/basefinal_tb_limp_enriched/observation_batch_2')

In [3]:
df_1.count()

723996919

In [ ]:
import os
path = '/data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/scripts_omop_tb/basefinal_tb_limp_enriched/observation_batch'
files = os.listdir(path)

files_to_upload = [f'{path}/{f}' for f in files if f.endswith('.parquet')]

for f in files_to_upload:
    print(f'Writing {f}')
    write_to_postgresl(spark.read.parquet(f).filter(F.col('observation_date').isNotNull()), tb_name='tuberculose.observation', write_mode='append')
    print(f'Written {f}')

Writing /data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/scripts_omop_tb/basefinal_tb_limp_enriched/observation_batch/part-00008-92a22ed2-57c6-48c9-8185-d5bfbef06df2-c000.snappy.parquet


Written /data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/scripts_omop_tb/basefinal_tb_limp_enriched/observation_batch/part-00008-92a22ed2-57c6-48c9-8185-d5bfbef06df2-c000.snappy.parquet
Writing /data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/scripts_omop_tb/basefinal_tb_limp_enriched/observation_batch/part-00013-92a22ed2-57c6-48c9-8185-d5bfbef06df2-c000.snappy.parquet


Written /data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/scripts_omop_tb/basefinal_tb_limp_enriched/observation_batch/part-00013-92a22ed2-57c6-48c9-8185-d5bfbef06df2-c000.snappy.parquet
Writing /data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/scripts_omop_tb/basefinal_tb_limp_enriched/observation_batch/part-00007-92a22ed2-57c6-48c9-8185-d5bfbef06df2-c000.snappy.parquet


Written /data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/scripts_omop_tb/basefinal_tb_limp_enriched/observation_batch/part-00007-92a22ed2-57c6-48c9-8185-d5bfbef06df2-c000.snappy.parquet
Writing /data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/scripts_omop_tb/basefinal_tb_limp_enriched/observation_batch/part-00005-92a22ed2-57c6-48c9-8185-d5bfbef06df2-c000.snappy.parquet


Written /data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/scripts_omop_tb/basefinal_tb_limp_enriched/observation_batch/part-00005-92a22ed2-57c6-48c9-8185-d5bfbef06df2-c000.snappy.parquet
Writing /data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/scripts_omop_tb/basefinal_tb_limp_enriched/observation_batch/part-00012-92a22ed2-57c6-48c9-8185-d5bfbef06df2-c000.snappy.parquet


### Salvando no Postgresql

In [ ]:
write_to_postgresl(df_observation, tb_name='tuberculose.observation', write_mode='append')